# Shor's Algorithm & Quantum Advantage for NP-Hard Problems

This notebook demonstrates:
1. Shor's algorithm for integer factorization
2. Quantum Fourier Transform applications
3. Connections to MWIS and combinatorial optimization

In [ ]:
# Install required packages
!pip install qiskit networkx numpy matplotlib sympy

In [ ]:
import sys
sys.path.append('..')

from src.shor.algorithm import ShorSimulator
from src.shor.mwis_connection import QuantumAdvantageAnalyzer, QuantumPeriodicMWIS
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import time

## Part 1: Shor's Algorithm Demonstration

In [ ]:
# Initialize Shor simulator
shor = ShorSimulator(n_qubits=10)

# Factor classic quantum computing examples
numbers = [15, 21, 35, 143]

for N in numbers:
    print(f"\nFactoring N = {N}:")
    print("-" * 30)
    
    start = time.time()
    p, q = shor.factor(N, verbose=True)
    elapsed = time.time() - start
    
    print(f"Result: {p} × {q} = {N}")
    print(f"Time: {elapsed:.3f} seconds")

In [ ]:
# Demonstrate breaking small RSA numbers
print("\n" + "="*60)
print("RSA BREAKING DEMONSTRATION")
print("="*60)

import time
shor = ShorSimulator(n_qubits=10)

# Create a custom demonstrate_rsa_break function since it might not exist
def demonstrate_rsa_break(shor, bits=16):
    """Demonstrate breaking small RSA-like numbers"""
    import random
    import math
    
    # Generate two small primes
    def is_prime(n, k=10):
        if n < 2:
            return False
        for p in [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]:
            if n % p == 0:
                return n == p
        # Miller-Rabin
        d = n - 1
        s = 0
        while d % 2 == 0:
            d //= 2
            s += 1
        for _ in range(k):
            a = random.randint(2, n - 2)
            x = pow(a, d, n)
            if x == 1 or x == n - 1:
                continue
            for _ in range(s - 1):
                x = (x * x) % n
                if x == n - 1:
                    break
            else:
                return False
        return True
    
    # Find primes
    p = None
    q = None
    while True:
        candidate = random.getrandbits(bits // 2)
        candidate |= (1 << (bits // 2 - 1)) | 1
        if is_prime(candidate):
            if p is None:
                p = candidate
            else:
                q = candidate
                break
    
    N = p * q
    print(f"Generated primes: p = {p}, q = {q}")
    print(f"Public key N = {p} × {q} = {N}")
    print(f"N has {N.bit_length()} bits")
    
    # Factor with Shor
    print(f"\nRunning Shor's algorithm...")
    start_time = time.time()
    p_found, q_found = shor.factor(N, verbose=True)
    elapsed = time.time() - start_time
    
    print(f"\nFactorization successful in {elapsed:.3f} seconds!")
    print(f"Found factors: {p_found} × {q_found} = {N}")
    print(f"Original factors were: {p} × {q}")
    
    return N, p_found, q_found, elapsed

N, p, q, elapsed = demonstrate_rsa_break(shor, bits=16)

In [ ]:
# Quantum circuit complexity analysis
print("\nQuantum Circuit Resource Requirements:")
print("-" * 50)

test_numbers = [15, 143, 2047, 9991]
for N in test_numbers:
    complexity = shor.get_circuit_complexity(N)
    print(f"\nN = {N} ({N.bit_length()} bits):")
    print(f"  Qubits needed: {complexity['total_qubits']}")
    print(f"  Gate estimate: {complexity['gate_count_estimate']:,}")
    print(f"  Time complexity: {complexity['time_complexity']}")

## Part 2: Quantum Advantage for MWIS

In [ ]:
# Create different graph structures
analyzer = QuantumAdvantageAnalyzer()

print("Graph Structure Analysis for Quantum Advantage")
print("="*60)

# Create three types of graphs
graphs = {
    "Regular (High Symmetry)": nx.random_regular_graph(3, 16),
    "Random (Low Symmetry)": nx.erdos_renyi_graph(16, 0.3),
    "Scale-free": nx.barabasi_albert_graph(16, 2)
}

for name, graph in graphs.items():
    # Add weights
    for node in graph.nodes():
        graph.nodes[node]['weight'] = np.random.uniform(0.5, 2.0)
    
    print(f"\n{name}:")
    analysis = analyzer.analyze_periodic_structures(graph)
    
    print(f"  Symmetry score: {analysis['symmetry_score']:.3f}")
    print(f"  Suggested quantum approach: {analysis['suggested_approach']}")

In [ ]:
# Visualize graphs
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, (name, graph) in enumerate(graphs.items()):
    ax = axes[idx]
    pos = nx.spring_layout(graph)
    nx.draw(graph, pos, ax=ax, with_labels=True, node_color='lightblue', node_size=300)
    ax.set_title(f"{name}\nSymmetry: {analyzer.analyze_periodic_structures(graph)['symmetry_score']:.3f}")

plt.tight_layout()
plt.show()

In [ ]:
# Compare quantum approaches
print("Quantum Algorithm Comparison for MWIS")
print("="*60)

solver_qft = QuantumPeriodicMWIS(use_qft=True)
solver_standard = QuantumPeriodicMWIS(use_qft=False)

test_graph = graphs["Regular (High Symmetry)"]

print("\n1. With QFT acceleration (inspired by Shor):")
result_qft = solver_qft.solve(test_graph)
print(f"   Algorithm: {result_qft['algorithm']}")
print(f"   Best weight: {result_qft['best_weight']:.2f}")
print(f"   Qubits needed: {result_qft['quantum_resources']['qubits']}")

print("\n2. Standard quantum optimization:")
result_std = solver_standard.solve(test_graph)
print(f"   Algorithm: {result_std['algorithm']}")
print(f"   Best weight: {result_std['best_weight']:.2f}")
print(f"   Qubits needed: {result_std['quantum_resources']['qubits']}")

## Part 3: Quantum Fourier Transform Visualization

In [ ]:
from src.shor.algorithm import QuantumFourierTransform

# Demonstrate QFT on simple states
print("Quantum Fourier Transform Demonstration")
print("="*50)

# Create QFT for 3 qubits
qft = QuantumFourierTransform(3)

# Test with computational basis states
states = ['|000⟩', '|100⟩', '|101⟩']
state_vectors = [
    np.array([1, 0, 0, 0, 0, 0, 0, 0], dtype=complex),  # |000⟩
    np.array([0, 0, 0, 0, 1, 0, 0, 0], dtype=complex),  # |100⟩
    np.array([0, 0, 0, 0, 0, 1, 0, 0], dtype=complex)   # |101⟩
]

for state_name, state_vec in zip(states, state_vectors):
    print(f"\nInput state: {state_name}")
    
    # Apply QFT
    transformed = qft.apply(state_vec)
    
    # Find largest amplitudes
    amplitudes = np.abs(transformed)**2
    top_indices = np.argsort(amplitudes)[-3:][::-1]
    
    print("  Top output states:")
    for idx in top_indices:
        state_bits = format(idx, '03b')
        prob = amplitudes[idx]
        print(f"    |{state_bits}⟩: probability = {prob:.3f}")

In [ ]:
# Show period finding visualization
print("\nPeriod Finding with QFT")
print("-" * 40)

# Simulate periodic function f(x) = a^x mod N
def periodic_function(a, N, q):
    """Create quantum state encoding periodic function"""
    state = np.zeros(q, dtype=complex)
    for x in range(q):
        f_x = pow(a, x, N)
        state[f_x] += 1/np.sqrt(q)
    return state / np.linalg.norm(state)

# Example from Shor's algorithm: a=7, N=15, period r=4
a, N = 7, 15
q = 32  # Number of points

periodic_state = periodic_function(a, N, q)

# Apply QFT
qft_8 = QuantumFourierTransform(5)  # 2^5 = 32
fourier_state = qft_8.apply(periodic_state)

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Original periodic function
ax1.bar(range(q), np.abs(periodic_state)**2)
ax1.set_xlabel('x')
ax1.set_ylabel('Probability')
ax1.set_title(f'Periodic Function: 7^x mod 15\n(Period r=4)')
ax1.grid(True, alpha=0.3)

# After QFT
ax2.bar(range(q), np.abs(fourier_state)**2)
ax2.set_xlabel('Frequency k')
ax2.set_ylabel('Probability')
ax2.set_title('After Quantum Fourier Transform\n(Peaks at multiples of q/r)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Insight:")
print("The QFT transforms periodic information in the amplitude")
print("into frequency peaks that can be measured classically.")
print("This is the core quantum advantage in Shor's algorithm!")

## Part 4: Practical Implications for NP-Hard Problems

In [ ]:
print("Quantum Advantage Summary")
print("="*60)

print("\nFrom Shor's Algorithm to Combinatorial Optimization:")
print("1. Exponential Speedup: O(exp(n)) → O(poly(n)) for factoring")
print("2. Key Technique: Quantum Fourier Transform")
print("3. Application to MWIS: Look for periodic structures in problems")
print("4. When it works: Problems with hidden symmetry/periodicity")
print("5. Current Status: Full Shor requires fault-tolerant quantum computers")

print("\n\nNear-term Applications (NISQ era):")
print("• Variational Quantum Algorithms (QAOA for MWIS)")
print("• Quantum annealing for optimization")
print("• Quantum machine learning for problem decomposition")
print("• Hybrid quantum-classical approaches")

print("\n\nRepository Structure for Client Review:")
print("1. src/shor/algorithm.py - Full Shor implementation")
print("2. src/shor/mwis_connection.py - Quantum advantage analysis")
print("3. notebooks/shor_demo.ipynb - This interactive demo")
print("4. src/mwis.py - MWIS quantum solvers")
print("5. requirements.txt - All dependencies")

print("\n\nTo run in cloud:")
print("1. Open in Google Colab: https://colab.research.google.com/github/shellworlds/shorfin/blob/main/notebooks/shor_demo.ipynb")
print("2. Or launch Binder: https://mybinder.org/v2/gh/shellworlds/shorfin/main")
print("3. No local installation needed!")

## Part 5: Live Cloud Execution Test

In [ ]:
# Test that everything works in cloud environment
print("Cloud Environment Test")
print("="*50)

print("\nChecking imports...")
imports_ok = True
try:
    import qiskit
    print(f"✓ Qiskit {qiskit.__version__} imported")
except ImportError as e:
    print(f"✗ Qiskit import failed: {e}")
    imports_ok = False

try:
    import networkx
    print(f"✓ NetworkX {networkx.__version__} imported")
except ImportError as e:
    print(f"✗ NetworkX import failed: {e}")
    imports_ok = False

try:
    import numpy
    print(f"✓ NumPy {numpy.__version__} imported")
except ImportError as e:
    print(f"✗ NumPy import failed: {e}")
    imports_ok = False

print(f"\nEnvironment check: {"PASSED" if imports_ok else "FAILED"}")

# Quick performance test
if imports_ok:
    print("\nRunning quick performance test...")
    start = time.time()
    
    # Create a simple MWIS instance
    graph = nx.erdos_renyi_graph(8, 0.4)
    for node in graph.nodes():
        graph.nodes[node]['weight'] = np.random.uniform(0.5, 2.0)
    
    # Analyze
    analyzer = QuantumAdvantageAnalyzer()
    analysis = analyzer.analyze_periodic_structures(graph)
    
    elapsed = time.time() - start
    print(f"✓ Performance test completed in {elapsed:.3f} seconds")
    print(f"  Graph symmetry score: {analysis['symmetry_score']:.3f}")

## Part 6: Client Review Checklist

In [ ]:
print("Client Review Checklist")
print("="*50)

checklist = [
    ("✅", "Repository created and populated", "https://github.com/shellworlds/shorfin"),
    ("✅", "Shor's algorithm implementation", "src/shor/algorithm.py"),
    ("✅", "MWIS quantum advantage analysis", "src/shor/mwis_connection.py"),
    ("✅", "Interactive Jupyter notebook", "notebooks/shor_demo.ipynb"),
    ("✅", "Cloud execution ready", "Colab/Binder links work"),
    ("✅", "Dependencies managed", "requirements.txt"),
    ("🔲", "Client added as collaborator", "Run: curl -X PUT ..."),
    ("🔲", "Performance benchmarks documented", "Add benchmark results"),
    ("🔲", "Test suite execution", "Run: python -m pytest tests/"),
    ("🔲", "Documentation complete", "README.md updated"),
]

for status, item, details in checklist:
    print(f"{status} {item:50} {details}")

print("\nNext steps for client:")
print("1. Review the repository structure")
print("2. Run the notebook in Colab (no installation needed)")
print("3. Test the algorithms with different inputs")
print("4. Review the quantum advantage analysis for MWIS")
print("5. Provide feedback on implementation")

print("\nQuestions for discussion:")
print("• How can quantum techniques be applied to your specific problems?")
print("• What size problems are you interested in solving?")
print("• Do you have existing classical solutions to compare against?")
print("• What are your performance requirements?")

print("\nContact for questions:")
print("• GitHub Issues: https://github.com/shellworlds/shorfin/issues")
print("• Email: [Your email address]")